In [1]:
import numpy as np
import os
import shutil

In [4]:
sed = '1'
linelist = 'linelist'
model = "hden"
hdens = np.arange(1., 13., 1.)
stopping_column_densities = np.arange(21., 25., 0.5)

for hden in hdens:

    model_name = f'hden:{hden}'

    # create directory if it doesn't already exist
    if not os.path.isdir(model_name):
        os.mkdir(model_name)

    # copy over sed
    shutil.copyfile(f'{sed}.sed', f'{model_name}/1.sed')

    # copy over linelist
    shutil.copyfile(f'{linelist}.dat', f'{model_name}/linelist.dat')

    run_commands = ['export CLOUDY_DATA_PATH="/Users/sw376/Dropbox/Research/software/cloudy/c23.01/data/:./" \n']

    for i, stopping_column_density in enumerate(stopping_column_densities):

        run_commands.append(f'/Users/sw376/Dropbox/Research/software/cloudy/c23.01/source/cloudy.exe -r {i+1} &\n')

        input = f"""table SED "1.sed" 
abundances "solar_GASS10.abn"
ionization parameter = -1.0 log
cosmic rays, background
hden {hden} log
iterate to convergence
set temperature floor 100 linear
stop temperature 500K
stop column density {stopping_column_density}
save overview  "{i+1}.ovr" last
save last abundances "{i+1}.abundances"
save last continuum "{i+1}.cont" units Angstroms
save line list column absolute last units angstroms "{i+1}.intrinsic_elin" "linelist.dat"
save line list emergent column absolute last units angstroms "{i+1}.emergent_elin" "linelist.dat"
save last lines, array "{i+1}.lines" units Angstroms no clobber
    """

        open(f'{model_name}/{i+1}.in', 'w').write(input)

    open(f'{model_name}/run.sh', 'w').writelines(run_commands)